In [27]:
import pandas as pd
import numpy as np
import opendatasets as od
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import joblib

In [3]:
# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/competitions/playground-series-s4e9/data'
# Using opendatasets let's download the data sets
od.download(dataset )

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  rajabhi1


Your Kaggle Key:

  ································


100%|█████████████████████████████████████████████████████████████████████████████| 7.84M/7.84M [00:02<00:00, 3.83MB/s]



Extracting archive .\playground-series-s4e9/playground-series-s4e9.zip to .\playground-series-s4e9


In [30]:
df_train = pd.read_csv("playground-series-s4e9/train.csv",index_col=0)
df_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [59]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [31]:
def extractHP(data):
    a = data.str.split(' ')
    return float(a[0][0][:-2])
def extractfuel(data):    
    a = data.str.split(' ')
    return float(a[0][1][:-1])
def extractCylinder(data):
    a = data.str.split(' ')
    return float(a[0][2])

In [32]:
df_train['HP'] = extractHP(df_train['engine'])
df_train['CylinderCap'] = extractfuel(df_train['engine'])
df_train['NoOfCylinder'] = extractCylinder(df_train['engine'])
df_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,HP,CylinderCap,NoOfCylinder
id,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,172.0,1.6,4.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,172.0,1.6,4.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,172.0,1.6,4.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,172.0,1.6,4.0


In [62]:
df_train.drop(columns=['engine'],inplace= True)
df_train.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,HP,CylinderCap,NoOfCylinder
id,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,172.0,1.6,4.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900,172.0,1.6,4.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,172.0,1.6,4.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500,172.0,1.6,4.0


In [63]:
df_train.drop(columns=['clean_title'],inplace= True)
df_train.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,price,HP,CylinderCap,NoOfCylinder
id,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,4200,172.0,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,4999,172.0,1.6,4.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,13900,172.0,1.6,4.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,45000,172.0,1.6,4.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,97500,172.0,1.6,4.0


In [36]:
# fuel_type, accident
print(df_train['accident'].isna().sum())
x = df_train['accident'].mode()[0]
df_train['accident'].fillna(x, inplace = True )
print(df_train['accident'].isna().sum())

2452
0


In [37]:
print(df_train['fuel_type'].isna().sum())
x = df_train['fuel_type'].mode()[0]
df_train['fuel_type'].fillna(x, inplace = True )
print(df_train['fuel_type'].isna().sum())

5083
0


In [83]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   brand         188533 non-null  object 
 1   model         188533 non-null  object 
 2   model_year    188533 non-null  int64  
 3   milage        188533 non-null  int64  
 4   fuel_type     188533 non-null  object 
 5   transmission  188533 non-null  object 
 6   ext_col       188533 non-null  object 
 7   int_col       188533 non-null  object 
 8   accident      188533 non-null  object 
 9   price         188533 non-null  int64  
 10  HP            188533 non-null  float64
 11  CylinderCap   188533 non-null  float64
 12  NoOfCylinder  188533 non-null  float64
dtypes: float64(3), int64(3), object(7)
memory usage: 24.2+ MB


In [95]:
df_train.to_csv('WorkingColumns.csv')

In [118]:
df_train = pd.read_csv("WorkingColumns.csv",index_col=0)

In [38]:
y_train = df_train['price']
X_train =  df_train.drop(['price'] , axis= 1)

In [39]:
X_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,HP,CylinderCap,NoOfCylinder
id,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,172.0,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,172.0,1.6,4.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,172.0,1.6,4.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,172.0,1.6,4.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,172.0,1.6,4.0


In [40]:
cat_col= ['brand' , 'model' , 'fuel_type', 'transmission' , 'ext_col', 'int_col', 'accident']
xCat = X_train[cat_col]
oe = OrdinalEncoder()
xCat =  oe.fit_transform(xCat)

In [129]:
type(xCat)

numpy.ndarray

In [127]:
X_train.drop(cat_col, axis = 1, inplace= True)
X_train.head()

,model_year,milage,HP,CylinderCap,NoOfCylinder
id,,,,,
0,2007,213000,172.0,1.6,4.0
1,2002,143250,172.0,1.6,4.0
2,2002,136731,172.0,1.6,4.0
3,2017,19500,172.0,1.6,4.0
4,2021,7388,172.0,1.6,4.0


In [131]:
df2 = pd.DataFrame(xCat, columns = cat_col)
X_train = pd.concat([X_train, df2], axis=1)
X_train.head()

,model_year,milage,HP,CylinderCap,NoOfCylinder,brand,model,fuel_type,transmission,ext_col,int_col,accident
0,2007,213000,172.0,1.6,4.0,31.0,495.0,2.0,38.0,312.0,71.0,1.0
1,2002,143250,172.0,1.6,4.0,28.0,930.0,2.0,38.0,263.0,10.0,0.0
2,2002,136731,172.0,1.6,4.0,9.0,1575.0,1.0,38.0,38.0,71.0,1.0
3,2017,19500,172.0,1.6,4.0,16.0,758.0,2.0,49.0,29.0,14.0,1.0
4,2021,7388,172.0,1.6,4.0,36.0,1077.0,2.0,23.0,29.0,10.0,1.0


In [132]:
X_train.to_csv('NumericalData.csv')

In [133]:
y_train.to_csv('Price.csv')

In [2]:
X_train , y_train = pd.read_csv('NumericalData.csv', index_col= 0), pd.read_csv('Price.csv', index_col = 0)

## Model building

In [3]:
# Applying random search for Random forest

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [20, 50, 100]

# Minimum number of samples required at each leaf node
min_samples_leaf = [10, 20, 40]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
clf_rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
model_RF = RandomizedSearchCV(estimator = clf_rf, param_distributions = random_grid, n_iter = 6, cv = 3, verbose=1, random_state=0, n_jobs = -1,scoring = "neg_root_mean_squared_error" )
# Fit the random search model
model_RF.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


D:\Anacoda\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=6, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [10, 20, 40],
                                        'min_samples_split': [20, 50, 100],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=0, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [29]:
#save your model or results
joblib.dump(model_RF, 'RandomForestBestModel.pkl')

#load your model for further usage
# joblib.load("model_file_name.pkl")


['RandomForestBestModel.pkl']

In [4]:
X_train.shape

(188533, 12)

In [5]:
X_train.head()

,model_year,milage,HP,CylinderCap,NoOfCylinder,brand,model,fuel_type,transmission,ext_col,int_col,accident
0,2007,213000,172.0,1.6,4.0,31.0,495.0,2.0,38.0,312.0,71.0,1.0
1,2002,143250,172.0,1.6,4.0,28.0,930.0,2.0,38.0,263.0,10.0,0.0
2,2002,136731,172.0,1.6,4.0,9.0,1575.0,1.0,38.0,38.0,71.0,1.0
3,2017,19500,172.0,1.6,4.0,16.0,758.0,2.0,49.0,29.0,14.0,1.0
4,2021,7388,172.0,1.6,4.0,36.0,1077.0,2.0,23.0,29.0,10.0,1.0


In [4]:
model_RF.best_score_

-73208.90654845502

In [5]:
model_RF.best_estimator_

RandomForestRegressor(max_depth=60, max_features='sqrt', min_samples_leaf=20,
                      min_samples_split=50, n_estimators=800)

## Testing 

In [23]:
df_test = pd.read_csv("playground-series-s4e9/test.csv")
df_test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [7]:
df_test.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title'],
      dtype='object')

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [9]:
df_test.drop(columns='id', inplace=True, axis =1)
df_test.reset_index()
df_test.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [10]:
def extractHP(data):
    a = data.str.split(' ')
    return float(a[0][0][:-2])
def extractfuel(data):    
    a = data.str.split(' ')
    return float(a[0][1][:-1])
def extractCylinder(data):
    a = data.str.split(' ')
    return float(a[0][2])

In [11]:
df_test['HP'] = extractHP(df_test['engine'])
df_test['CylinderCap'] = extractfuel(df_test['engine'])
df_test['NoOfCylinder'] = extractCylinder(df_test['engine'])
df_test.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,HP,CylinderCap,NoOfCylinder
0,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,240.0,2.0,4.0
1,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes,240.0,2.0,4.0
2,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN,240.0,2.0,4.0
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN,240.0,2.0,4.0
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,240.0,2.0,4.0


In [12]:
df_test.drop(columns=['engine'],inplace= True)
df_test.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,HP,CylinderCap,NoOfCylinder
0,Land,Rover LR2 Base,2015,98000,Gasoline,6-Speed A/T,White,Beige,None reported,Yes,240.0,2.0,4.0
1,Land,Rover Defender SE,2020,9142,Hybrid,8-Speed A/T,Silver,Black,None reported,Yes,240.0,2.0,4.0
2,Ford,Expedition Limited,2022,28121,Gasoline,10-Speed Automatic,White,Ebony,None reported,NaN,240.0,2.0,4.0
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,Automatic,Silician Yellow,Black,None reported,NaN,240.0,2.0,4.0
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,A/T,Gray,Black,None reported,Yes,240.0,2.0,4.0


In [13]:
df_test.drop(columns=['clean_title'],inplace= True)
df_test.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,HP,CylinderCap,NoOfCylinder
0,Land,Rover LR2 Base,2015,98000,Gasoline,6-Speed A/T,White,Beige,None reported,240.0,2.0,4.0
1,Land,Rover Defender SE,2020,9142,Hybrid,8-Speed A/T,Silver,Black,None reported,240.0,2.0,4.0
2,Ford,Expedition Limited,2022,28121,Gasoline,10-Speed Automatic,White,Ebony,None reported,240.0,2.0,4.0
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,Automatic,Silician Yellow,Black,None reported,240.0,2.0,4.0
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,A/T,Gray,Black,None reported,240.0,2.0,4.0


In [14]:
print(df_test['accident'].isna().sum())
x = df_test['accident'].mode()[0]
df_test['accident'].fillna(x, inplace = True )
print(df_test['accident'].isna().sum())
print(df_test['fuel_type'].isna().sum())
x = df_test['fuel_type'].mode()[0]
df_test['fuel_type'].fillna(x, inplace = True )
print(df_test['fuel_type'].isna().sum())

1632
0
3383
0


In [17]:
cat_col= ['brand' , 'model' , 'fuel_type', 'transmission' , 'ext_col', 'int_col', 'accident']
tCat = df_test[cat_col]
oe = OrdinalEncoder()
tCat = oe.fit_transform(tCat)

In [18]:
df_test.drop(cat_col, axis = 1, inplace= True)
df_test.head()

,model_year,milage,HP,CylinderCap,NoOfCylinder
0,2015,98000,240.0,2.0,4.0
1,2020,9142,240.0,2.0,4.0
2,2022,28121,240.0,2.0,4.0
3,2016,61258,240.0,2.0,4.0
4,2018,59000,240.0,2.0,4.0


In [19]:
dfT = pd.DataFrame(tCat, columns = cat_col)
df_test = pd.concat([df_test, dfT], axis=1)
df_test.head()

,model_year,milage,HP,CylinderCap,NoOfCylinder,brand,model,fuel_type,transmission,ext_col,int_col,accident
0,2015,98000,240.0,2.0,4.0,26.0,1388.0,2.0,16.0,302.0,10.0,1.0
1,2020,9142,240.0,2.0,4.0,26.0,1375.0,3.0,31.0,261.0,14.0,1.0
2,2022,28121,240.0,2.0,4.0,14.0,636.0,2.0,3.0,302.0,57.0,1.0
3,2016,61258,240.0,2.0,4.0,3.0,182.0,2.0,39.0,259.0,14.0,1.0
4,2018,59000,240.0,2.0,4.0,3.0,181.0,2.0,38.0,127.0,14.0,1.0


In [20]:
df_test.to_csv('testData.csv')

In [21]:
y_pred = model_RF.predict(df_test)

In [24]:
xId = df_test['id']

In [25]:
dfT = pd.DataFrame(y_pred, columns = ['Price'])
temp = pd.concat([xId,dfT] , axis =1)
temp.set_index('id', inplace= True)
temp.to_csv('PredictedPrice.csv')

In [26]:
temp.head()

,Price
id,
188533,20465.237155
188534,81774.977501
188535,60583.304392
188536,26411.996563
188537,32110.819559
